# # Regression SHAP MMP

In [1]:
%load_ext autoreload

### Import libraries

In [ ]:
import os 
from ML.machine_learning_models import *
from ML.machine_learning_models import Model_Evaluation as ml_evaluation
from ML.data_preprocessing import Dataset
from ML.ml_utils_reg import create_directory, ECFP4, set_global_determinism
from ML.data_utils import *
from tqdm.notebook import tqdm
from ML.utils_shap import get_df_shap
import joblib
import shap
%autoreload 2

### Parameters

In [ ]:
# parameters
# ML models
model_list = ['SVR', 'RFR']
# number of trials
trial_splits = 10
# number of mmp trials
mmp_trials = 1
# fingerprint
fingerprint = 'ECFP4'
# approach
approach = 'regression'

### Load data

In [ ]:
# dataset path
db_path = "./dataset/"
main_folder = "./regression_shap_mmp/"
results_path = f'./{main_folder}/{fingerprint}/{approach}/'

In [ ]:
df_regression = pd.read_csv(db_path + "chembl_33_pIC50.csv")
df_regression

### Load MMP datasets

In [ ]:
mmp_path = "./ccrlib_master/"
df_mmp = pd.read_csv(mmp_path + "df_mmp_final_top10.csv")
df_mmp

In [ ]:
df_shapL = []

for target in tqdm(df_mmp.tid.unique()[:]):

    target_path = create_directory(f'./{main_folder}/{fingerprint}/{approach}/{target}/')
    
    print(f"Target: {target}")

    df_regression_target = df_regression[df_regression.tid == target].reset_index(drop=True)
    
    for mmp_trial in range(mmp_trials):
        print(f"MMP trial: {mmp_trial}")
        
        df_mmp_target = df_mmp[(df_mmp.tid == target) & (df_mmp.mmp_trial == mmp_trial)]
        
        for split in ['Random', 'Stratified']:
            print(f"Split: {split}")
            for trial in range(trial_splits):
                print(f"Trial: {trial}")
                
                # train test split
                regression_set, train_idx, test_idx = split_data(df_regression_target, df_mmp_target, test_size=0.5, random_seed=trial, split_type=split)

                # dataset 
                dataset = Dataset(np.array(ECFP4(regression_set.smiles.values)), np.array(regression_set.pPot.values))
                dataset.add_instance("target", regression_set.tid.values)
                dataset.add_instance("smiles", regression_set.smiles.values)
                dataset.add_instance("cid", regression_set.cid.values)
                dataset.add_instance("mmp_id", regression_set.mmp_id.values)
                dataset.add_instance("analog_series_id", regression_set.analog_series_id.values)
                dataset.add_instance("train_test", regression_set.train_test.values)
                dataset.add_instance("similarity", regression_set.similarity.values)
                dataset.add_instance("dPot", regression_set.dpot.values)
                
                # Training set
                training_set = dataset[train_idx]
                # Test set
                test_set = dataset[test_idx]
                
                # Set seed
                set_global_determinism(seed=trial)
                
                for model in model_list:
                    print(f"Model: {model}")
                    # Create saving directory
                    model_fpath = create_directory(f"./regression_mmp/{fingerprint}/{approach}/{target}/{split}/{model}/", verbose=False)
    
                    # ml model
                    model_loaded = None
                    if os.path.exists(os.path.join(model_fpath, f"{model}_{trial}_{mmp_trial}.pkl")):
                        ml_model = joblib.load(os.path.join(model_fpath, f"{model}_{trial}_{mmp_trial}.pkl"))
                        print(f"Model {model}_{trial} loaded")
                        model_loaded = True
                    else:
                        ml_model = MLModel(training_set, model)
                        ml_model = ml_model.model
                        # save model
                        joblib.dump(ml_model, os.path.join(model_fpath, f"{model}_{trial}_{mmp_trial}.pkl"))
    
                    # Model evaluation
                    model_eval_train = ml_evaluation(model=ml_model, data=training_set, model_id=model,
                                                         model_loaded=model_loaded)
                        
                    model_eval_test = ml_evaluation(model=ml_model, data=test_set, model_id=model,
                                                    model_loaded=model_loaded)
                    
                    # Prediction df
                    predictions_train = get_df_results(model_eval_train.predictions, trial=trial, approach=approach, 
                                                        fingerprint=fingerprint, split=split, mm_trial=mmp_trial)

                    # Prediction df
                    predictions_test = get_df_results(model_eval_test.predictions, trial=trial, approach=approach, 
                                                      fingerprint=fingerprint, split=split, mm_trial=mmp_trial)

                    # SHAP
                    if model == 'RFR':
                        model_expl_name = 'TreeExplainer'
                        model_explainer = shap.TreeExplainer(ml_model, feature_perturbation='interventional', data=training_set.features)
                        expected_value = model_explainer.expected_value
                        shap_values_tr = model_explainer.shap_values(X=training_set.features, check_additivity=True).tolist()
                        shap_values_te = model_explainer.shap_values(X=test_set.features, check_additivity=True).tolist()
                        pred_train = ml_model.predict(training_set.features)
                        pred_test = ml_model.predict(test_set.features)

                    elif model == 'SVR':
                        model_expl_name = 'SVETA'
                        model_explainer = ml_model
                        shap_values_tr = [model_explainer.vector_feature_weights(vec) for vec in csr_matrix(training_set.features)]
                        shap_values_te = [model_explainer.vector_feature_weights(vec) for vec in csr_matrix(test_set.features)]
                        expected_value = model_explainer.expected_value[0]
                        pred_train = ml_model.predict(csr_matrix(training_set.features))
                        pred_test = ml_model.predict(csr_matrix(test_set.features))

                    # shap df
                    df_shap_trial_tr = get_df_shap(data=training_set, trial=trial, model=model, split=split, expected_value=expected_value,
                                                   model_explainer=model_expl_name, predictions_test=pred_train, shap_values=shap_values_tr)

                    df_shap_trial_te = get_df_shap(data=test_set, trial=trial, model=model, split=split, expected_value=expected_value,
                                                   model_explainer=model_expl_name, predictions_test=pred_test, shap_values=shap_values_te)
                    
                    df_shap_trial_tr['train_test'] = 'train'
                    df_shap_trial_te['train_test'] = 'test'
                    df_shap_trial_tr['mmp_trial'] = mmp_trial
                    df_shap_trial_te['mmp_trial'] = mmp_trial
                    df_shap_trial = pd.concat([df_shap_trial_tr, df_shap_trial_te])
                    # save df_shap per trial
                    df_shap_trial.to_pickle(os.path.join(target_path, f"df_shap_{model}_{trial}_{mmp_trial}.pkl"))
    
                    df_shapL.append(df_shap_trial)
                    
                    del ml_model, df_shap_trial

df_shap = pd.concat(df_shapL)
df_shap.to_pickle(os.path.join(results_path, f'df_shap.pkl'))
del df_shapL, df_shap